In [1]:
from pydantic import BaseModel, Field
from typing import List, Optional
from fastapi import FastAPI, File, UploadFile

import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn import functional as F
from torch import nn
import torchmetrics
import torchvision
from tqdm import tqdm
import transformers
import open_clip

from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2

import warnings
import matplotlib.pyplot as plt
import os
import numpy as np
import requests
import csv

In [2]:
MEAN = (0.4914, 0.4822, 0.4465)
STD = (0.2023, 0.1994, 0.2010)
device = 'cuda'
data_dir = 'data_sirius'
save_dataset_dir = 'saved_dataset_data'

In [3]:
class LogoDataset(Dataset):
    def __init__(self,
        mode,
        data_dir,
        fraction: float = 0.7,
        logo_classifier_thr = 0.4,
        transform=None,
    ):
        ## pipeline to zero-shot images
        model_name = "ViT-g-14"
        pretrained = "laion2b_s34b_b88k"
        label_classifier, _, preprocess = open_clip.create_model_and_transforms(model_name, pretrained=pretrained)
        tokenizer = open_clip.get_tokenizer(model_name)
        label_classifier.to(device).eval()

        prompt = [
            "a white shield with black letter T logo"
            "a yellow shield with black letter T logo",
            "a gray shield with letter black T logo",
            "a stylized black letter T in a yellow shield",
            "a black T logo on yellow background",
            "a stylized black letter T in a white shield",
            "a black T logo on white background",
            "a stylized black letter T in a gray shield",
            "a black T logo on gray background",
            "no T logo present"
        ]

        text_tokens = tokenizer(prompt).to(device)

        ## list of tuples: (img_path, label)
        self._items = []

        ## will be used later for augmentations
        self._transform = transform

        ## we can't store all the images in memory at the same time,
        ## because sometimes we have to work with very large datasets
        ## so we will only store data paths
        ## (also this is convenient for further augmentations)
        all_img_paths = []
        all_y = []
        
        file_names = os.listdir(data_dir)
        pbar = tqdm(range(len(file_names)), desc="Processing dataset items")

        for img_name in file_names:
            img_path = f"{data_dir}/{img_name}"
            img = Image.open(img_path).convert("RGB")

            if not (img.width > 10 and img.height > 10):
                pbar.update(1)
                continue

            img_input = preprocess(img).unsqueeze(0).to(device)

            with torch.no_grad():
                img_features = label_classifier.encode_image(img_input)
                text_features = label_classifier.encode_text(text_tokens)

            img_features /= img_features.norm(dim=-1, keepdim=True)
            text_features /= text_features.norm(dim=-1, keepdim=True)

            logits = 100.0 * img_features @ text_features.T
            probs = logits.softmax(dim=-1)

            top_probs, top_labels = probs[0].topk(2)
            print(img_path, prompt[top_labels[0]], top_probs[0].item())

            all_img_paths.append(img_path)
            # all_y.append(1 if len(detected_logos) > 0 else 0) 
            all_y.append(1)

            pbar.update(1)

        train_size = int(fraction * len(all_img_paths))
        test_size = len(all_img_paths) - train_size

        if mode == "train":
            for obj_ind in range(train_size):
                self._items.append((all_img_paths[obj_ind], all_y[obj_ind]))
        elif mode == "test":
            for obj_ind in range(test_size):
                self._items.append(
                    (all_img_paths[train_size + obj_ind], all_y[train_size + obj_ind])
                )

    def __len__(self):
        return len(self._items)

    def __getitem__(self, index):
        img_path, label = self._items[index]

        ## read image
        image = Image.open(img_path).convert("RGB")
        label = np.copy(label).astype("float")

        ## make augmentation for image
        if self._transform:
            pass

        return image, label
    
    def save(self, path):
        """
        Save image paths and its labels into file to minimize init time
        
        Args:
            path: Path to save dataset

        Returns:
            None 
        """

        with open(path, "w", newline="") as csv_file:
            writer = csv.writer(csv_file)
            writer.writerow(['path', 'label'])
            writer.writerows(self._items)

In [ ]:
## pipeline to zero-shot images

model_name = "ViT-g-14"
pretrained = "laion2b_s34b_b88k"
label_classifier, _, preprocess = open_clip.create_model_and_transforms(model_name, pretrained=pretrained)
tokenizer = open_clip.get_tokenizer(model_name)
label_classifier.to(device).eval()

# prompt = [
#     "a white shield with black letter T logo"
#     "a yellow shield with black letter T logo",
#     "a gray shield with letter black T logo",
#     "a stylized black letter T in a yellow shield",
#     "a black T logo on yellow background",
#     "a stylized black letter T in a white shield",
#     "no T logo present"
# ]

prompt = [
      'a white triangular shield badge',
      'a yellow triangular shield badge',
      'a gray triangular shield badge',
      'a black triangular shield badge',
      'no triangle shield on the photo',
]

text_tokens = tokenizer(prompt).to(device)

file_names = os.listdir(data_dir)
all_items = []

pbar = tqdm(range(len(file_names)), desc="Processing dataset items")

for img_name in file_names:
    img_path = f"{data_dir}/{img_name}"
    img = Image.open(img_path).convert("RGB")

    if not (img.width > 10 and img.height > 10):
        pbar.update(1)
        continue

    img_input = preprocess(img).unsqueeze(0).to(device)

    with torch.no_grad():
        img_features = label_classifier.encode_image(img_input)
        text_features = label_classifier.encode_text(text_tokens)

    img_features /= img_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    logits = 100.0 * img_features @ text_features.T
    probs = logits.softmax(dim=-1)

    top_probs, top_labels = probs[0].topk(2)
    print(img_path, prompt[top_labels[0]], top_probs[0].item())

    all_items.append((img_path, not (prompt[top_labels[0]] != len(prompt) - 1)))

    pbar.update(1)

with open(save_dataset_dir, "w", newline="") as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(['path', 'label'])
    writer.writerows(all_items)

Processing dataset items:   0%|          | 2/30079 [00:00<55:12,  9.08it/s]

data_sirius/0003065d1258b85b191e64b569a5f9ae.jpg no triangle shield on the photo 0.8858017325401306
data_sirius/0004019ef5c13431099af96214e3e5d4.jpg no triangle shield on the photo 0.7509791254997253


Processing dataset items:   0%|          | 5/30079 [00:00<48:23, 10.36it/s]

data_sirius/000520dee7c145b59fe71e23c6b5d97b.jpg a black triangular shield badge 0.9805811643600464
data_sirius/0009746f722ddd70b68f24319e68ed35.jpg a gray triangular shield badge 0.5712555050849915
data_sirius/000b155124d0b69b7a87329b839506ba.jpg no triangle shield on the photo 0.5124070644378662


Processing dataset items:   0%|          | 7/30079 [00:00<46:56, 10.68it/s]

...

data_sirius/077cde59cb6a4e1184687783bcbcddc9.jpg a black triangular shield badge 0.8585981726646423
data_sirius/077e6ed226bd8d27821d13e6b2a2d992.jpg a white triangular shield badge 0.5390108227729797
data_sirius/077eeae2b3e81d6269d683079b1ae050.jpg a yellow triangular shield badge 0.9160966277122498


Processing dataset items:   3%|▎         | 866/30079 [01:10<47:11, 10.32it/s]

data_sirius/077fb2234ba738c4a8cbe88af41cde24.jpg a gray triangular shield badge 0.7801409363746643
data_sirius/078101e8cde73b13230b7e5a58debe76.jpg a black triangular shield badge 0.38847824931144714
data_sirius/07821c7554b5e5524a8a957acb371675.jpg a yellow triangular shield badge 0.7359873652458191
data_sirius/078a2bb52dcd99300ef0b41afcb17c99.jpg a black triangular shield badge 0.8982284069061279


KeyboardInterrupt: 

Processing dataset items:   3%|▎         | 867/30079 [01:26<47:10, 10.32it/s]

In [ ]:
ds_train = LogoDataset(mode='train', data_dir='data_sirius')
ds_test = LogoDataset(mode='test', data_dir='data_sirius')

ds_train.save('saved_dataset_data/train.csv')
ds_test.save('saved_dataset_data/test.csv')


open_clip_model.safetensors:   0%|          | 0.00/5.47G [00:00<?, ?B/s]